# Advanced Mapping

So far we've built our maps using the top-level mapping functions.
These functions are useful for tutorials, but don't give us the full flexibility that we might need when working with arbitrary Python functions.
They're also sometimes inconvenient to use, especially if you don't like typing the names of your functions over and over.
The tools described in this tutorial fix those problems.

## Starmap

So far, we've been carefully avoiding any functions that needed to be mapped over keyword arguments, or that had more than one positional argument.
When we did use a keyword argument, in [Working With Files](working-with-files.ipynb), we noted that [htmap.map](../api.rst#htmap.map) applies the same keyword arguments to each component.
[htmap.starmap](../api.rst#htmap.starmap) provides the flexibility to completely specify the positional and keyword arguments for every component.

Unfortunately, that looks like this:

In [1]:
import htmap

def power(x, p = 1):
        return x ** p

In [2]:
starmap = htmap.starmap(
    func = power,
    args = [
        (1,),
        (2,),
        (3,),
    ],
    kwargs = [
        {'p': 1},
        {'p': 2},
        {'p': 3},
    ],
)

print(list(starmap))  # [1, 4, 27]

[1, 4, 27]


A slightly more pleasant but less obvious way to construct the arguments would be like this:

In [3]:
starmap = htmap.starmap(
    func = power,
    args = ((x,) for x in range(1, 4)),
    kwargs = ({'p': p} for p in range(1, 4)),
)

print(list(starmap))  # [1, 4, 27]

[1, 4, 27]


But that isn't really a huge improvement.
Sometimes you'll need the power and compactness of `starmap`, but we recommend using...

## Map Builders

A map builder lets you build a map piece-by-piece.
To get a map builder, call [htmap.build_map](../api.rst#htmap.build_map) as a context manager, then call it as if it were the mapped function itself:

In [4]:
with htmap.build_map(func = power) as builder:
    for x in range(1, 4):
        builder(x, p = x)

map = builder.map
print(list(map))  # [1, 4, 27]

[1, 4, 27]


The map builder catches the function calls and turns them into a map.
The map is created when the `with` block ends, and at that point you can grab the actual [htmap.Map](../api.rst#htmap.Map) from the builder's `.map` attribute.

## Mapped Functions

If you're tired of re-typing the name of your function all the time, create a [htmap.MappedFunction](../api.rst#htmap.MappedFunction) using the [htmap.mapped](../api.rst#htmap.mapped) decorator:

In [5]:
@htmap.mapped
def double(x):
    return 2 * x

print(double)

<MappedFunction(func = <function double at 0x7f0a79564510>, map_options = {})>


The resulting `MappedFunction` has methods that correspond to all the mapping functions, but with the function already filled in.

For example:

In [6]:
doubled = double.map(range(10))

print(list(doubled))  # [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


The real utility of mapped functions is that they can carry default map options, which are **inherited** by any maps created from them.
For example, we could have written the file comparison function from [Working With Files](working-with-files.ipynb) like this:

In [7]:
@htmap.mapped(map_options = htmap.MapOptions(fixed_input_files = ['master.txt']))
def compare_files(test_file, master_file = None):
    test = Path(test_file)
    master = Path(master_file)
    
    return test.read_text() == master.read_text()

Now we always transfer the master file, without needing to specify it in the `MapOptions` of the actual `map` call.

See [htmap.MapOptions](../api.rst#htmap.MapOptions) for some notes about how these inherited map options behave.

In the [next tutorial](error-handling.ipynb) we'll finally address the most important part of programming: what to do when things go wrong!